In [1]:
library(data.table)
library(dplyr, quietly = T)
library(RMySQL, quietly = T)

Warning message:
“package ‘dplyr’ was built under R version 3.4.4”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
“package ‘RMySQL’ was built under R version 3.4.4”Warning message:
“package ‘DBI’ was built under R version 3.4.4”

## Connect to TCRD to get protein.ids and corresponding ENSGs

In [2]:
DBHOST <- 'localhost'
DBNAME <- 'tcrd6'
DBUSER <- 'smathias'

dbconn <- dbConnect(MySQL(), host=DBHOST, dbname=DBNAME, user=DBUSER)
sql <- "SELECT p.id AS protein_id, x.value AS ensg 
        FROM protein p, xref x 
        WHERE p.id = x.protein_id AND x.xtype = 'ENSG'"
prots <- dbGetQuery(dbconn, sql)
dbDisconnect(dbconn)
rm(dbconn)
setDT(prots) # convert data.frame to data.table
head(prots)

[1] TRUE

protein_id,ensg
19154,ENSG00000173598
19154,ENSG00000177144
12316,ENSG00000211637
12222,ENSG00000211638
12315,ENSG00000211639
12255,ENSG00000211642


## Download and read input file

In [3]:
#download.file("http://www.proteinatlas.org/download/normal_tissue.tsv.zip", destfile = "../data/HPA/normal_tissue.tsv.zip")

hpa <- fread("7z x -so ../data/HPA/normal_tissue.tsv.zip", header = T, sep = "\t", quote = "", na.strings = "")
head(hpa)

Gene,Gene name,Tissue,Cell type,Level,Reliability
ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved
ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved
ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved
ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved
ENSG00000000003,TSPAN6,breast,adipocytes,Not detected,Approved
ENSG00000000003,TSPAN6,breast,glandular cells,High,Approved


In [4]:
nrow(hpa)

[1] 1053330

## Join hpa and prots on ENSGs

In [5]:
hpa <- merge(hpa, prots, by.x = "Gene", by.y = "ensg")
head(hpa)

Gene,Gene name,Tissue,Cell type,Level,Reliability,protein_id
ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved,9720
ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved,9720
ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved,9720
ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved,9720
ENSG00000000003,TSPAN6,breast,adipocytes,Not detected,Approved,9720
ENSG00000000003,TSPAN6,breast,glandular cells,High,Approved,9720


In [6]:
nrow(hpa)

[1] 1049601

## Cleanup
### First, there are tissues like 'skin 1' and 'skin 2', etc. Get rid of the numbers

In [7]:
hpa[, Tissue := sub("\\s\\d+$","", Tissue)]

### Combine `Tissue` and `Cell type` into one field

In [8]:
hpa <- mutate(hpa, Tissue = paste(Tissue, `Cell type`, sep = " - "))
hpa$`Cell type` <- NULL
head(hpa)

Gene,Gene name,Tissue,Level,Reliability,protein_id
ENSG00000000003,TSPAN6,adrenal gland - glandular cells,Not detected,Approved,9720
ENSG00000000003,TSPAN6,appendix - glandular cells,Medium,Approved,9720
ENSG00000000003,TSPAN6,appendix - lymphoid tissue,Not detected,Approved,9720
ENSG00000000003,TSPAN6,bone marrow - hematopoietic cells,Not detected,Approved,9720
ENSG00000000003,TSPAN6,breast - adipocytes,Not detected,Approved,9720
ENSG00000000003,TSPAN6,breast - glandular cells,High,Approved,9720


In [9]:
unique(hpa$Reliability)
table(hpa$Level)

[1] "Approved"  "Uncertain" "Enhanced"  "Supported"


        High          Low       Medium Not detected 
      119229       181352       291486       457534 

### Get rid of rows with Uncertain Reliability

In [10]:
hpa <- filter(hpa, Reliability != "Uncertain")
setDT(hpa)
nrow(hpa)
table(hpa$Level)
table(hpa$Reliability)

[1] 846061


        High          Low       Medium Not detected 
      100359       138362       228135       379205 


 Approved  Enhanced Supported 
   430705    296905    118451 

### Convert Reliabliity and Level columns into ordered factors

In [11]:
hpa[, Level := factor(x = Level, levels = c("Not detected", "Low", "Medium", "High"), ordered = T)]
hpa[, Reliability := factor(x = Reliability, levels = c("Enhanced", "Supported", "Approved"), ordered = T)]
nrow(hpa)
table(hpa$Level)
table(hpa$Reliability)

[1] 846061


Not detected          Low       Medium         High 
      379205       138362       228135       100359 


 Enhanced Supported  Approved 
   296905    118451    430705 

## Group by protein_id and Tissue; then order (descending) by Reliability and Level keeping only the most reliable value for a given protein_id/Tissue

In [12]:
#hpa <- hpa[1:1000,]
hpa <- hpa[, head(.SD[order(-Reliability, -Level)], 1), by = .(protein_id, Tissue)]
head(hpa)
nrow(hpa)

protein_id,Tissue,Gene,Gene name,Level,Reliability
9720,adrenal gland - glandular cells,ENSG00000000003,TSPAN6,Not detected,Approved
9720,appendix - glandular cells,ENSG00000000003,TSPAN6,Medium,Approved
9720,appendix - lymphoid tissue,ENSG00000000003,TSPAN6,Not detected,Approved
9720,bone marrow - hematopoietic cells,ENSG00000000003,TSPAN6,Not detected,Approved
9720,breast - adipocytes,ENSG00000000003,TSPAN6,Not detected,Approved
9720,breast - glandular cells,ENSG00000000003,TSPAN6,High,Approved


[1] 782711

In [13]:
hpa_tau <- function(exps) {
  exps$Level2 <- numeric(nrow(exps))
  exps$Level2[exps$Level == "Not detected"] <- 0
  exps$Level2[exps$Level == "Low"] <- 1
  exps$Level2[exps$Level == "Medium"] <- 2
  exps$Level2[exps$Level == "High"] <- 3
  #exps$Level2 <- as.numeric(exps$Level2)
  exps <- group_by(exps, Tissue) %>% summarize(Level2 = median(Level2))  
  tau <- sum(1-(exps$Level2/max(exps$Level2)))/(length(unique(exps$Tissue)) - 1)
  return(tau)
}

hpa.tau <- group_by(hpa, Gene) %>% do(TAU = hpa_tau(.))
hpa.tau$TAU <- unlist(hpa.tau$TAU)
nrow(hpa.tau)
head(hpa.tau)

[1] 10498

Gene,TAU
ENSG00000000003,0.6392694
ENSG00000000419,0.3288288
ENSG00000000457,0.5571429
ENSG00000000938,0.9178082
ENSG00000000971,NaN
ENSG00000001084,0.6027397


In [15]:
hpa <- merge(hpa, hpa.tau, by.x = "Gene", by.y = "Gene")
nrow(hpa)
head(hpa)

[1] 782711

Gene,protein_id,Tissue,Gene name,Level,Reliability,TAU
ENSG00000000003,9720,adrenal gland - glandular cells,TSPAN6,Not detected,Approved,0.6392694
ENSG00000000003,9720,appendix - glandular cells,TSPAN6,Medium,Approved,0.6392694
ENSG00000000003,9720,appendix - lymphoid tissue,TSPAN6,Not detected,Approved,0.6392694
ENSG00000000003,9720,bone marrow - hematopoietic cells,TSPAN6,Not detected,Approved,0.6392694
ENSG00000000003,9720,breast - adipocytes,TSPAN6,Not detected,Approved,0.6392694
ENSG00000000003,9720,breast - glandular cells,TSPAN6,High,Approved,0.6392694


## Write output file

In [16]:
OUTPUT_FILE <- '../data/HPA/HPA.tsv'

if (file.exists(OUTPUT_FILE)) {
  file.remove(OUTPUT_FILE)
}
fwrite(hpa, file = OUTPUT_FILE, quote = T, sep = "\t", col.names = T, row.names = F, na = "None")

[1] TRUE